In [1]:
import os
os.getcwd() 
os.chdir(os.pardir)
os.getcwd()



'/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning'

In [6]:
import torch
from src.comet_models import LatentEBM, LatentEBM128
from src.config.load_config import load_config
from src.dataloader import MRI2D
import logging
from torch.utils.data import DataLoader
from src.test_final import gen_image as gen_image_our

# Test

In [7]:


def get_last_recon(checkpoint_path, model_type):
    config_path = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/config/2DMRI_config.yml'
    dataset_type = 'MRI2D'

    run_name = checkpoint_path.split("/")[-1].split(".")[0]
    logging.info(f"Run name: {run_name}")
    # Load configuration
    config = load_config(config_path)
    logging.info(f"Loaded config: {config}")

    run_name = checkpoint_path.split("/")[-1].split(".")[0]
    logging.info(f"Run name: {run_name}")

    dataset = MRI2D(config)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    if model_type == "LatentEBM":
        model_class = LatentEBM
    elif model_type == "LatentEBM128":
        model_class = LatentEBM128
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    state_dicts = checkpoint if isinstance(checkpoint, list) else [checkpoint]
    models = [model_class(config, dataset_type).to(device) for _ in range(len(state_dicts))]
    for i, model in enumerate(models):
        model.load_state_dict(state_dicts[i], strict=False)
        model.eval()  # Set to evaluation mode


    # Prepare data
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=config.data_workers)
    im, _ = next(iter(dataloader))
    im = im.to(device)

    # Embed latent
    latent = models[0].embed_latent(im)
    latents = torch.chunk(latent, config.components, dim=1)
    # each latent is a component, and has size (batch_size, latent_dim)

    im_neg = torch.rand_like(im)
    im_negs = gen_image_our(latents, config, models, im_neg, 30)

    our_final_image = im_negs[-1][0].detach().cpu().numpy().transpose(1, 2, 0)
    print(our_final_image.shape)

    return our_final_image



In [8]:
checkpoint_ebm128 = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/models/latentEBM128_UN_767.pth'
latentebm128_image = get_last_recon(checkpoint_ebm128, 'LatentEBM128')

checkpoint_ebm = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/models/latentEBM_old_UN768.pth'
latentebm_image = get_last_recon(checkpoint_ebm, 'LatentEBM')


/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/.venv/lib/python3.9/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


RuntimeError: Error(s) in loading state_dict for LatentEBM128:
	size mismatch for conv1.weight: copying a param with shape torch.Size([8, 1, 3, 3]) from checkpoint, the shape in current model is torch.Size([4, 1, 3, 3]).
	size mismatch for conv1.bias: copying a param with shape torch.Size([8]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for gain.weight: copying a param with shape torch.Size([8, 64]) from checkpoint, the shape in current model is torch.Size([4, 64]).
	size mismatch for gain.bias: copying a param with shape torch.Size([8]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for bias.weight: copying a param with shape torch.Size([8, 64]) from checkpoint, the shape in current model is torch.Size([4, 64]).
	size mismatch for bias.bias: copying a param with shape torch.Size([8]) from checkpoint, the shape in current model is torch.Size([4]).
	size mismatch for layer_encode.conv1.weight: copying a param with shape 

# LatentEBM128

In [ ]:
checkpoint_ebm128 = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/models/latentEBM128_UN_767.pth'

config_path = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/config/2DMRI_config.yml'
dataset_type = 'MRI2D'

run_name = checkpoint_ebm128.split("/")[-1].split(".")[0]
logging.info(f"Run name: {run_name}")
# Load configuration
config = load_config(config_path)
logging.info(f"Loaded config: {config}")

run_name = checkpoint_ebm128.split("/")[-1].split(".")[0]
logging.info(f"Run name: {run_name}")

dataset = MRI2D(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load checkpoint
checkpoint = torch.load(checkpoint_ebm128, map_location=device)
state_dicts = checkpoint if isinstance(checkpoint, list) else [checkpoint]
models = [LatentEBM128(config, dataset_type).to(device) for _ in range(len(state_dicts))]
for i, model in enumerate(models):
    model.load_state_dict(state_dicts[i], strict=False)
    model.eval()  # Set to evaluation mode


# Prepare data
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=config.data_workers)
im, _ = next(iter(dataloader))
im = im.to(device)

# Embed latent
latent = models[0].embed_latent(im)
latents = torch.chunk(latent, config.components, dim=1)
# each latent is a component, and has size (batch_size, latent_dim)

im_neg = torch.rand_like(im)
im_negs = gen_image_our(latents, config, models, im_neg, 30)

our_final_image = im_negs[-1][0].detach().cpu().numpy().transpose(1, 2, 0)
print(our_final_image.shape)


